In [1]:
import tensorflow as tf
import numpy as np


In [2]:
saver = tf.train.import_meta_graph("/tmp/Ch11Ex70.9904650710254913.ckpt.meta")

In [3]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

last = tf.get_default_graph().get_operation_by_name("DNN/hiddenLayers/Elu_4")
training = tf.get_default_graph().get_tensor_by_name("PlaceholderWithDefault:0")

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

train_five_nine_indicies = mnist.train.labels > 4

ytrain59 = mnist.train.labels[train_five_nine_indicies]
Xtrain59 = mnist.train.images[train_five_nine_indicies]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
Xm = tf.placeholder(tf.float32 , shape=(None , 100) , name="newX")
ym = tf.placeholder(tf.int64 , shape=(None) , name="newY")

logitsnew = tf.layers.dense(Xm , 5 , activation=None)

loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=(ym-5) , logits=logitsnew)

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss)

acc = tf.reduce_mean(tf.cast(tf.math.in_top_k(logitsnew , ym-5 ,1) , tf.float64))

init = tf.global_variables_initializer()

In [11]:
n_epoch = 2
batch_size = 50
n_batches = Xtrain59.shape[0] // batch_size

with tf.Session() as sess:
    saver.restore(sess , "/tmp/Ch11Ex70.9904650710254913.ckpt")
    sess.run(init)
    Xcatched = last.outputs[0].eval(feed_dict={X: Xtrain59 , training : False})
    XcatchedBatches = np.array_split(Xcatched , n_batches)
    ybatches = np.array_split(ytrain59 , n_batches)
    for Xbatch , ybatch in zip(XcatchedBatches , ybatches):
        sess.run(training_op , feed_dict={Xm:Xbatch , ym:ybatch})
    saver.save(sess , "/tmp/Ch11Ex8.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/Ch11Ex70.9904650710254913.ckpt


In [7]:
Xcatched.shape

(26962, 100)

In [16]:
with tf.Session() as sess:
    saver.restore(sess , "/tmp/Ch11Ex8.ckpt")
    sess.run(init)
    accuracy = logitsnew.eval(feed_dict={Xm:Xcatched , ym:ytrain59})

INFO:tensorflow:Restoring parameters from /tmp/Ch11Ex8.ckpt


In [21]:
accuracy[1]

array([ 0.06403241, -0.06435296, -0.56984246, -0.30129725,  0.00242251],
      dtype=float32)

In [20]:
ytrain59

array([7, 6, 8, ..., 5, 6, 8], dtype=uint8)